In [ ]:
import json
import numpy
import os
import pandas
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [ ]:
# 1. % incoming transfer packages with Wholesale type
# 2. % incoming transfer packages with wholesale price
# 3. % inventory packages with incoming transfer package
# 4. % inventory packages with incoming transfer package with wholesale price

In [ ]:
def _format_as_percent(number):
    return f'{round(number * 100, 2)}%'

def run_analysis_for_company(company_name, company_identifier, company_licenses):
    company_incoming_transfer_packages_for_analysis_query = create_queries.create_company_incoming_transfer_packages_for_analysis_query(
        company_identifier,
        company_licenses
    )
    company_inventory_packages_for_analysis_query = create_queries.create_company_inventory_packages_for_analysis_query(
        company_identifier,
        company_licenses
    )
    company_incoming_transfer_packages_for_analysis_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_for_analysis_query, engine)
    company_inventory_packages_for_analysis_dataframe = pandas.read_sql_query(company_inventory_packages_for_analysis_query, engine)

    incoming_transfer_packages_dataframe = company_incoming_transfer_packages_for_analysis_dataframe
    inventory_packages_dataframe = company_inventory_packages_for_analysis_dataframe
    
    count_incoming_package_transfers = incoming_transfer_packages_dataframe.count()['package_id']
    count_accepted_incoming_package_transfers = incoming_transfer_packages_dataframe[incoming_transfer_packages_dataframe['shipment_package_state'] == 'Accepted'].count()['package_id']
    
    count_incoming_packages = len(incoming_transfer_packages_dataframe['package_id'].unique())

    incoming_packages_dataframe = incoming_transfer_packages_dataframe.drop_duplicates(subset=['package_id'], keep='last')
    incoming_packages_dataframe = incoming_packages_dataframe[incoming_transfer_packages_dataframe['shipment_package_state'] == 'Accepted']
    
    count_wholesale_incoming_packages = incoming_packages_dataframe[incoming_packages_dataframe['shipment_transaction_type'] == 'Wholesale'].count()['package_id']
    count_with_wholesale_price_incoming_packages = incoming_packages_dataframe.count()['receiver_wholesale_price']

    count_inventory_packages = inventory_packages_dataframe.count()['package_id']
    join_data_frame = inventory_packages_dataframe.merge(incoming_packages_dataframe, how='inner', on='package_id')
    join_data_frame_count_result = join_data_frame.count()
    
    count_with_incoming_transfer_package_inventory_packages = join_data_frame_count_result['package_id']
    count_with_incoming_wholesale_price_inventory_packages = join_data_frame_count_result['receiver_wholesale_price']

    result = [
        company_name,
        company_identifier,
        count_incoming_package_transfers,
        _format_as_percent(count_accepted_incoming_package_transfers / count_incoming_package_transfers), # % incoming package transfers that are Accepted
        count_incoming_packages,
        _format_as_percent(count_wholesale_incoming_packages / count_incoming_packages), # % incoming packages that are Wholesale
        _format_as_percent(count_with_wholesale_price_incoming_packages / count_incoming_packages), # % incoming packages that have wholesale price
        count_inventory_packages,
        _format_as_percent(count_with_incoming_transfer_package_inventory_packages / count_inventory_packages), # % inventory packages with matching incoming package
        _format_as_percent(count_with_incoming_wholesale_price_inventory_packages / count_inventory_packages) # % inventory packages with matching incoming package that have wholesale price
    ]
    return result

In [ ]:
company_tuples = [
    ('Boston Bud Factory', 'BBF', 'MA', ['MR281525']),
    ('Cali Greens LLC', 'CG', 'CA', ['C9-0000157-LIC']),
    ('California Patients Club', 'CPC', 'CA', ['C9-0000056-LIC']),
    ('California Street Cannabis', 'CSC', 'CA', ['C10-0000670-LIC']),
    ("California's Choice Collective", 'CCC', 'CA', ['C12-0000087-LIC']),
    ('Calm and Collective', 'CC', 'CA', ['C10-0000758-LIC']),
    ('Diem Cannabis MA', 'DCM', 'MA', ['MR282376', 'MR283369']),
    ('Diem Cannabis OR', 'DCO', 'OR', ['050-10052885D4C', '050-10070593E9E']),
    ('Embarc', 'EM', 'CA', ['C10-0000695-LIC', 'C10-0000774-LIC', 'C10-0000786-LIC']),
    ('Eskar Holdings', 'EH', 'CA', ['MR283073']),
    ('Good Tree', 'GT', 'CA', ['C9-0000170-LIC', 'C9-0000385-LIC']),
    ('Green Rose Green Leaf Care', 'GRG', 'CA', ['C10-0000408-LIC', 'C10-0000169-LIC']),
    ('Grupo Flor', 'GF', 'CA', ['C10-0000805-LIC', 'C10-0000224-LIC', 'C10-0000700-LIC', 'C10-0000824-LIC', 'C10-0000117-LIC', 'C12-0000191-LIC']),
    ('Herbarium', 'HB', 'CA', ['C10-0000301-LIC', 'C10-0000624-LIC', 'C10-0000339-LIC']),
    ('Hueneme Patient Consumer Collective', 'HPCC', 'CA', ['C10-0000005-LIC', 'C10-0000064-LIC']),
    ('HumblCann Inc', 'HC', 'CA', ['C9-0000341-LIC']),
    ('Indacut', 'IDC', 'CA', ['C9-0000320-LIC']),
    ('Kaya Cannabis', 'KC', 'CO', ['402R-00002', '402R-00544', '402R-00900', '402R-00574', '402R-00035', '402R-00515', '402-01146', '402-00573', '402-00037', '402-00323', '402-00664']),
    ('MedLeaf', 'ML', 'CA', ['C9-0000300-LIC']),
    ('Medicine Woman', 'MW', 'CA', ['C10-0000332-LIC']),
    ('Monex Place Wellness', 'MPW', 'CA', ['C10-0000547-LIC']),
    ('Moonflower Delivery', 'MD', 'CA', ['C9-0000247-LIC']),
    ('NXTLVL Delivery', 'ND', 'CA', ['C9-0000427-LIC']),
    ('Purple Lotus', 'PL', 'CA', ['C10-0000241-LIC']),
    ('Quality Roots', 'QR', 'MI', ['AU-R-000233', 'AU-R-000346', 'PC-000356']),
    ('Royal Apothecary', 'RA', 'CA', ['C10-0000596-LIC']),
    ('Sovereign', 'SO', 'CA', ['C10-0000271-LIC', 'C12-0000328-LIC', 'C9-0000130-LIC']),
    ('Welcome The Healing Touch', 'WHT', 'CA', ['C10-0000548-LIC']),
]

In [ ]:
results = []

for company_name, company_identifier, us_state, company_licenses in company_tuples:
    company_result = run_analysis_for_company(company_name, company_identifier, company_licenses)
#     print(company_name, company_identifier)
#     print(company_result)
    results.append(company_result)

In [ ]:
columns = [
    'Company name',
    'Company identifier',
    '# incoming package transfers',
    '% accepted incoming package transfers',
    '# incoming packages',
    '% wholesale incoming packages',
    '% have price incoming packages',
    '# inventory packages',
    '% have incoming package inventory packages',
    '% have have price incoming package inventory packages',
]
retailer_companies_dataframe = pandas.DataFrame(results, columns=columns)
retailer_companies_dataframe